In [2]:
import sys

#Due to leftoevererrors in Nathan's python installation, some cleaning up occurs here
#sys.path.append("./code/")
#sys.path.remove('/usr/local/lib/python2.7/site-packages') 


import matplotlib.pyplot as plt
%matplotlib inline
import copy
import datetime
import pickle
import gzip
import os,glob
import time
import numpy as np
import pandas as pd
import operator
#sys.path

import regex as re
import datetime
import math
import time

import plotly.plotly as py
import plotly.graph_objs as go

########## to be able to plot offline (without sending the plots to the plotly server every time)
import plotly.offline as offline
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
################



from IPython.core.display import display,HTML
display(HTML("<style>.container { width:100% !important; }</style>"))  # to make the notebook use the entire width of the browser


In [5]:
master_dict_ref_UT_dict_traj = pickle.load(open('../data/test_trajectories_OLD.pkl', 'rb'))
print ("read master dict",len(master_dict_ref_UT_dict_traj.keys()))

read master dict 1956


10

In [6]:
def datetime_filler_traj(traj_dict):
    # Generate a new dictionary from a trajectory dictionary with datetime objects. Includes the plos_flag value
    

    numeric = re.compile('[0-9]+')
    alpha = re.compile('[A-Za-z]+')
    month_dict = {'JAN':1, 'FEB': 2, 'MAR':3, 'APR':4, 'MAY':5, 'JUN': 6, 
                  'JUL': 7, 'AUG': 8, 'SEP': 9, 'OCT':10, 'NOV': 11, 'DEC': 12,
                 'SPR': 3, 'SUM': 6, 'FAL': 9, 'WIN': 12}
    formatted_traj_dict = {}
    
    for ref_ut in traj_dict:
        formatted_traj_dict[ref_ut] = {}
        for cite_ut in traj_dict[ref_ut]:
            date = traj_dict[ref_ut][cite_ut][0]
            year = traj_dict[ref_ut][cite_ut][1]
            plos_flag = traj_dict[ref_ut][cite_ut][2]
            
            date_field = []

            if type(date) == str:
                
                for i in re.findall(alpha, date):
                    date_field.append(i)
                for i in re.findall(numeric, date):
                    date_field.append(i)
                
                if len(date_field) > 2: # No weirdness here, only 2 values max for date field
                    print(date_field)
                
                if len(date_field[0])==1: # There's a single date that is just '0'. Likely a database error/nan
                    print(date_field)

                    
            else:
                date_field = ['JUNE', '15']
                
            month = -1
            day = -1
            
            if date_field[0] in month_dict:
                month = month_dict[date_field[0]]
                if len(date_field)==2:
                    if len(date_field[1])<3:
                        day = int(date_field[1])
            
                        
            if not math.isnan(year):
                if month != -1:
                    if day != -1:
                        dt = datetime.datetime(year, month, day)
                    else:
                        dt = datetime.datetime(year, month, 15)
                    
                else:
                    dt = datetime.datetime(year, 6, 15)


            else:
                dt = -1
            
            formatted_traj_dict[ref_ut][cite_ut] = [dt, plos_flag]

    return formatted_traj_dict

In [7]:
dict_traject = datetime_filler_traj(master_dict_ref_UT_dict_traj)
print ("done converting datetimes")

with open('../data/test_trajectories_datetimes.pkl', 'wb') as handle:
    pickle.dump(dict_traject, handle, protocol = 2)
print ("written: ../data/test_trajectories_datetimes.pkl")  

done converting datetimes
written: ../data/test_trajectories_datetimes.pkl


000087539300015 415
A1987J391400011 5467
A1997XH86900011 218
A1997XX70200010 252
000082946400010 2944
000084023900023 315
000165228200045 2239
A1987J406700007 36467
A1991GM61300001 1365
A1996UB59500002 2655


In [8]:
len(master_dict_ref_UT_dict_traj['000082946400010'])

2944

In [9]:
# i create a simpler dictionary (because sorting it by value when the value is a tuple is problematic)
master_dict_traject={}
for llave1 in dict_traject:
    master_dict_traject[llave1]={}
   # print( dict_traject[llave1])
    for llave2 in dict_traject[llave1]:        
        master_dict_traject[llave1][llave2]=dict_traject[llave1][llave2][0]
    #print (llave1, len(master_dict_traject[llave1]))

 

### i create a new dict (ut as keys) of dicts: where dates are keys and values are total count of citations    
new_dict={}
for ut in master_dict_traject:
    print (ut, len(master_dict_traject[ut]))
   
    new_dict[ut]={}
    dict_citating_paper_date=master_dict_traject[ut]
       
    cont_cit =1
    sorted_dict = sorted(dict_citating_paper_date.items(), key=operator.itemgetter(1))
    for pair in sorted_dict:        
        date=pair[1]

#         try:                  
#             new_dict[ut][date] += cont_cit
#         except KeyError:
        new_dict[ut][date] = cont_cit
        cont_cit +=1
        
#     print (sum(new_dict_ut_dict_citation_dates_counts[ut].values()))
    #input()
    
    
with open('../data/dict_ut_dict_dates_citations_trajectories_datetimes.pkl', 'wb') as handle:
    pickle.dump(new_dict, handle, protocol = 2)
print ("written: ../data/dict_ut_dict_dates_citations_trajectories_datetimes.pkl")  

000087539300015 415
A1987J391400011 5467
A1997XH86900011 218
A1997XX70200010 252
000082946400010 2944
000084023900023 315
000165228200045 2239
A1987J406700007 36467
A1991GM61300001 1365
A1996UB59500002 2655
A1990ED16700008 46519
A1997XU79300002 42772
A1996TW56700049 1232
000083534200033 1020
A1997XT82700053 1316
A1967A307700002 1868
A1996UG35000008 339
000085750800017 351
A1971I616900007 3683
A1989T157400011 999
000085657600002 737
A1997XL49800004 533
A1954UL80800017 4133
A1983RQ64200021 5094
000087526300074 7402
000074153700010 1058
000084836600041 807
000083638400006 956
A1984AJZ6300006 1534
A1951UM01800005 4002
A1987K675600009 4793
000165296000031 4316
000079300100007 755
000165588300006 152
000089985400007 935
000074038600008 2254
A1977CY39700012 22734
000087923100003 1865
000072894400022 1808
A1975AR81900012 2560
000083512600003 1484
A1988Q102000008 659
A1981LG16200045 838
000084842000048 243
A1990EQ41300004 1943
A1991GV87700062 446
A1997XW77200029 271
A1993MK55500010 507
A1994PU1

A1995TR87100014 631
A1993MJ75700003 1272
A1995TF32200004 3861
000080552200002 930
000085742200011 1797
A1982NL32300028 6233
A1997XG41600054 781
A1997WG23500047 868
A1993ML91800002 549
000074705900057 1315
A1996UX27000009 1091
A1997WF29300011 98
A1972N215800001 10172
A1958WX09300012 43661
A1972N572600003 30911
A1997XE54900006 1363
000086179700003 1827
000076862300009 259
000084896300093 863
A1995TK18700028 1917
000165954000011 1567
A1997YD89800007 1464
A1986F013900001 17453
A1994QP05000016 6104
A1997XW69400016 16299
A1995RZ34200038 1926
A1997WA37300021 547
000084375400116 239
A1996UQ45500075 145
A1996VY97400056 345
000086727100044 357
A1989AK28500058 836
000078461700049 807
A1997XD97700029 3282
A1990CP51800002 5117
A1995TL18000014 2104
A1996TZ57700003 6926
000165225800069 653
000165766600007 780
000165728800033 1672
000079509000053 11775
A1987L491900007 2929
A1989AD04700030 834
A1996VF05900015 386
000071498400004 28538
A1997XK60100001 1169
000083229200015 1445
000078608900001 3297
A1982

A1977EE49800008 3829
A1992JT97700109 435
A1985ATP4300003 1549
A1985TZ64400005 4864
A1997YK98600035 881
000072421300021 3427
000165474100039 1660
A1997XJ99000005 1724
A1997WP05600040 2912
A1988N057400014 3189
A1994PA94400008 3188
A1988L777300011 3243
000086762300019 208
A1995RH81000016 298
A1958ZP88900021 3440
A1995QH21700005 4791
A1995RP75500001 5996
000083583900006 830
A1996VY92400012 600
A1984SW54000004 760
A1989AW74500037 1175
000084896300009 1339
A1995TH77500007 802
A1995TH77500008 733
000072970700010 1134
A1996TX60600004 881
A1996TX60600013 1855
000077338100049 740
A1952UH81500008 10757
A1993MH32200061 878
000073681800007 414
A1994PU26400005 213
000084488200012 603
000080726800004 412
A1993LU59200004 2676
A1992HJ14500004 695
A1945XX03600006 3494
000083193100006 331
A1997XY53900028 5041
A1994NL55500020 1028
A1994NW81600027 1793
000089998600020 5846
000085559200061 589
000090136700010 869
A1979JE30900003 2125
000165096200006 721
000081063800047 182
A1997WT56800057 565
A1995QA8820001

A1995RE89200001 248
000071923500046 584
000087732700052 677
000086049100051 964
000080809000042 849
A1996VW71200049 900
000083913600059 619
000076196400011 509
000083882600012 425
000075433800026 278
A1986A474400028 1128
000165053000003 1038
A1993LV24200006 1960
A1974S947500011 1431
000087980500017 702
000090071100013 1153
000088608000021 613
000080363400009 563
A1996TW70200019 731
000076727900008 671
A1994PD65700019 1348
A1994PA83200023 848
000081589400072 1508
000078399500016 1650
000080932700049 1383
A1995TE58300007 1907
000079615100014 2976
000087515900029 2063
000089907600011 3424
A1996VV78200006 596
A1964XF68900004 3745
A1995QU35800002 869
000072195600006 170
000087475100036 250
A1997XG74800066 4353
000074379400022 500
A1996VB68000004 440
A1997WJ84900001 323
000076852700054 1559
000078484900001 837
A1977CV05900021 798
A1991GP03800006 266
A1996VK03800004 318
A1997XB54600006 274
A1996VM68100018 986
000085654800015 511
000083976800011 358
A1997WK06900055 480
000075179800017 1006
A19

written: ../data/dict_ut_dict_dates_citations_trajectories_datetimes.pkl


In [3]:
new_dict = pickle.load(open('../data/dict_ut_dict_dates_citations_trajectories_datetimes_.pkl', 'rb'))
print ("read master dict",len(new_dict.keys()))



read master dict 9139


In [106]:
sorted_dict = sorted(master_dict_traject['A1991GM61300001'].items(), key=operator.itemgetter(1))
for pair  in sorted_dict:
    print (pair[1])

1991-06-15 00:00:00
1992-03-15 00:00:00
1992-04-15 00:00:00
1992-05-15 00:00:00
1992-05-19 00:00:00
1992-05-25 00:00:00
1992-06-15 00:00:00
1992-06-15 00:00:00
1992-07-01 00:00:00
1992-07-15 00:00:00
1992-07-15 00:00:00
1992-09-05 00:00:00
1992-09-05 00:00:00
1992-10-15 00:00:00
1992-10-15 00:00:00
1992-10-15 00:00:00
1992-10-15 00:00:00
1992-10-15 00:00:00
1992-10-15 00:00:00
1992-11-05 00:00:00
1992-11-24 00:00:00
1992-11-25 00:00:00
1992-12-01 00:00:00
1992-12-15 00:00:00
1992-12-15 00:00:00
1993-01-15 00:00:00
1993-02-01 00:00:00
1993-02-15 00:00:00
1993-02-26 00:00:00
1993-03-09 00:00:00
1993-03-15 00:00:00
1993-04-01 00:00:00
1993-04-01 00:00:00
1993-04-25 00:00:00
1993-04-25 00:00:00
1993-05-03 00:00:00
1993-05-15 00:00:00
1993-06-15 00:00:00
1993-06-15 00:00:00
1993-06-15 00:00:00
1993-06-15 00:00:00
1993-06-15 00:00:00
1993-06-15 00:00:00
1993-07-01 00:00:00
1993-07-01 00:00:00
1993-07-06 00:00:00
1993-08-05 00:00:00
1993-08-05 00:00:00
1993-08-15 00:00:00
1993-08-15 00:00:00


In [128]:
for date in sorted(new_dict['A1991GM61300001']):
    print (date, new_dict['A1991GM61300001'][date])

1991-06-15 00:00:00 1
1992-03-15 00:00:00 2
1992-04-15 00:00:00 3
1992-05-15 00:00:00 4
1992-05-19 00:00:00 5
1992-05-25 00:00:00 6
1992-06-15 00:00:00 8
1992-07-01 00:00:00 9
1992-07-15 00:00:00 11
1992-09-05 00:00:00 13
1992-10-15 00:00:00 19
1992-11-05 00:00:00 20
1992-11-24 00:00:00 21
1992-11-25 00:00:00 22
1992-12-01 00:00:00 23
1992-12-15 00:00:00 25
1993-01-15 00:00:00 26
1993-02-01 00:00:00 27
1993-02-15 00:00:00 28
1993-02-26 00:00:00 29
1993-03-09 00:00:00 30
1993-03-15 00:00:00 31
1993-04-01 00:00:00 33
1993-04-25 00:00:00 35
1993-05-03 00:00:00 36
1993-05-15 00:00:00 37
1993-06-15 00:00:00 43
1993-07-01 00:00:00 45
1993-07-06 00:00:00 46
1993-08-05 00:00:00 48
1993-08-15 00:00:00 50
1993-09-01 00:00:00 52
1993-09-05 00:00:00 53
1993-10-05 00:00:00 54
1993-10-08 00:00:00 55
1993-10-15 00:00:00 59
1993-11-01 00:00:00 60
1993-11-15 00:00:00 62
1993-11-29 00:00:00 63
1993-12-15 00:00:00 67
1994-01-01 00:00:00 68
1994-01-14 00:00:00 69
1994-01-18 00:00:00 70
1994-02-04 00:00:00

In [4]:
# df_trajectories= pd.DataFrame.from_dict(new_dict)#, orient='index')
# df_trajectories

# d={'col1':[1,2,3], 'col2':[11,22,33], 'col_id':[1,1,1]}
# df = pd.DataFrame(data=d)

# df = pd.DataFrame(columns=['A','B','C','D','E','F','G'])
# for i in range(5):
#     df = df.append({'A': i}, ignore_index=True)


lista_dict=[]
#df = pd.DataFrame(columns=['UT','date','citation_count'])
for ut in new_dict:
    #print (ut)
    for date in new_dict[ut]:
        count= new_dict[ut][date]
        
        #####  ojo! it is WAY faster to create the list of dictionaries first and then convert it into a dataframe, than add one row at a time!!
       # df = df.append({'UT': ut, 'date':date,'citation_count':count}, ignore_index=True)
        dict_aux={'UT': ut, 'date':date,'citation_count':count}
        lista_dict.append(dict_aux)
    
    


df_trajectories= pd.DataFrame.from_dict(lista_dict)


df_trajectories.to_pickle('/home/staff/julia/at_Northwestern/In_Text_Citations/In-Text-Citations-New/data/df_UT_dates_citation_counts.pickle')
df_trajectories 

000070935900017
000071002000055
000071035800006
000071055100011
000071069300004
000071078700008
000071108000068
000071111800005
000071113100003
000071147700006
000071148900016
000071149100006
000071149100031
000071149200005
000071159300003
000071169100016
000071181500004
000071182800069
000071182800071
000071182800102
000071209200001
000071236500013
000071259600022
000071262200011
000071265600025
000071265600038
000071275000008
000071281400006
000071281400013
000071281400019
000071281400020
000071281400021
000071290000007
000071295500022
000071326100042
000071326100050
000071326100052
000071326100053
000071326100055
000071351300003
000071368500022
000071370400050
000071380900057
000071408100042
000071409200008
000071429500062
000071429500065
000071440200067
000071440200074
000071445300002
000071446200036
000071450100014
000071451100027
000071473700010
000071484400033
000071484600001
000071494000021
000071494500001
000071498400004
000071501900013
000071570800037
000071570800045
00007157

000080200400038
000080200900001
000080203300016
000080207100032
000080220100007
000080222900021
000080223000005
000080227100001
000080230000006
000080246500038
000080246500069
000080246500083
000080246500094
000080251600023
000080267900020
000080271400025
000080299100011
000080322800005
000080326600011
000080326600017
000080335700043
000080335700049
000080344900019
000080359100037
000080363400006
000080363400009
000080386800007
000080408000008
000080409700009
000080416100040
000080416100060
000080427400035
000080427400059
000080429600033
000080430600055
000080430600056
000080430900005
000080434500003
000080439200034
000080442200006
000080444100055
000080451800001
000080488200030
000080488500004
000080497900008
000080527100012
000080527100054
000080527100072
000080528000008
000080532000017
000080532000019
000080547800050
000080547800061
000080548100035
000080548100039
000080550300002
000080552200002
000080556700002
000080559000005
000080573400009
000080573400012
000080590400002
00008060

000087257300043
000087270900048
000087270900050
000087270900053
000087291400057
000087296500010
000087296500013
000087318700051
000087318700056
000087318700059
000087318700067
000087318700078
000087318700086
000087332500014
000087351600030
000087353700019
000087353700029
000087354800016
000087357900010
000087357900021
000087358700048
000087360800007
000087362500008
000087375400010
000087375400011
000087382300032
000087386500003
000087387300050
000087396400006
000087438300041
000087446500001
000087446500018
000087449500056
000087454000013
000087454900006
000087459200007
000087459200018
000087459200025
000087459400002
000087459400016
000087459400019
000087475100036
000087475100039
000087484900015
000087501500043
000087503800041
000087503800042
000087503800053
000087504500011
000087504500012
000087508500011
000087515900029
000087526300047
000087526300049
000087526300074
000087526300076
000087526300095
000087526300118
000087527300008
000087528500009
000087532000020
000087532000024
00008753

A1972N198500011
A1972N215800001
A1972N251400036
A1972N572600003
A1972N660100004
A1972N729000003
A1972N768500015
A1972N770800027
A1972N802700001
A1972N947500002
A1972N951600009
A1972O022800002
A1972O089300047
A1972O315300017
A1972O335300018
A1972O391800004
A1973O337700022
A1973O408900003
A1973O606200035
A1973O812600010
A1973P145300001
A1973P199100055
A1973P284600006
A1973P354200008
A1973P396400004
A1973P453400020
A1973P513300019
A1973P573400003
A1973P608700016
A1973P663300005
A1973P698200005
A1973P772600003
A1973P842500029
A1973Q097000007
A1973Q138900013
A1973Q143000004
A1973Q203900021
A1973Q393700020
A1973Q457200016
A1973Q569400021
A1973Q627600001
A1973Q656900004
A1973Q658200002
A1973Q662300005
A1973Q950000001
A1973R076100003
A1973R135800019
A1973R188400011
A1973R229300031
A1973R353300001
A1973R469800002
A1973R469800003
A1973R487900025
A1973R588100010
A1973R622300021
A1973R637400010
A1973R686100011
A1973R799000012
A1973R836200009
A1973S338300007
A1974R802200001
A1974S275800007
A1974S38

A1985AMZ6700003
A1985ANC7000032
A1985ANC7100060
A1985ANS5300024
A1985ANV6400007
A1985APG0700042
A1985APH3400002
A1985APJ8100007
A1985APL1600011
A1985APN1700023
A1985APP1500020
A1985APQ2500004
A1985APU0100002
A1985APU0400026
A1985AQA2300013
A1985AQB4200014
A1985AQH8400010
A1985AQV5700003
A1985ARC2700001
A1985ARU6400009
A1985ARY7400006
A1985ASF4000009
A1985ASJ2400010
A1985ASK3400001
A1985ASL8200004
A1985ASR0500019
A1985ASS7900014
A1985AST7600020
A1985ATB3400002
A1985ATC5800002
A1985ATC6000050
A1985ATF7700005
A1985ATF7700022
A1985ATF8800008
A1985ATG7500053
A1985ATP4300003
A1985ATR2500009
A1985ATT2700011
A1985AUA8000007
A1985AUJ6000036
A1985AUP0700021
A1985AUR1800078
A1985AUU2900007
A1985AVC5100002
A1985AVF9100003
A1985AVG0100018
A1985AVR1200003
A1985AVV1000007
A1985AVV2800008
A1985AVW1700050
A1985AVX9700018
A1985AWB1600007
A1985AWC7800005
A1985AWS1000003
A1985AWV6100030
A1985AYH1800014
A1985TX79000001
A1985TY78300004
A1985TZ17000001
A1985TZ64400005
A1986A044500008
A1986A079600060
A1986A09

A1991FV17800075
A1991FV37300024
A1991FW16000027
A1991FW47300036
A1991FW91300013
A1991FY14900008
A1991FY28700007
A1991FY28800030
A1991FY48100009
A1991FY50200012
A1991FY98700043
A1991FZ25000040
A1991FZ62900017
A1991FZ96500003
A1991GA22600065
A1991GA30400038
A1991GA30400039
A1991GA94100007
A1991GA94100013
A1991GA94100019
A1991GB12200004
A1991GB21100046
A1991GB23700040
A1991GB31200002
A1991GB32600002
A1991GB63100011
A1991GB70100053
A1991GB70100077
A1991GB70100088
A1991GB72900001
A1991GB85100004
A1991GB89600003
A1991GB97700041
A1991GC06500002
A1991GC15200036
A1991GC15200046
A1991GC15300002
A1991GC74500006
A1991GC74500019
A1991GC74500020
A1991GD02000010
A1991GD03300001
A1991GD27400011
A1991GD39200003
A1991GD53300013
A1991GD96600001
A1991GD96600003
A1991GE05700013
A1991GE57100005
A1991GE74900005
A1991GE83100026
A1991GF09700045
A1991GF44500107
A1991GF68200002
A1991GF85100031
A1991GG31000002
A1991GG55200007
A1991GG96300027
A1991GH09600001
A1991GH32200007
A1991GH44500020
A1991GH48700003
A1991GH6

A1994MX64000015
A1994MX75900029
A1994MX77100023
A1994MX85400043
A1994MY10100016
A1994MY29600053
A1994MY31200008
A1994MY54100001
A1994MY57000037
A1994MY57300001
A1994MY84400016
A1994MY85400005
A1994MZ09200010
A1994MZ28500003
A1994MZ40000009
A1994MZ47400014
A1994MZ47400021
A1994MZ49800030
A1994MZ84900004
A1994NA02200005
A1994NA03000065
A1994NA17600008
A1994NA80600007
A1994NA87000046
A1994NA92100094
A1994NA97100005
A1994NB10200027
A1994NB10200028
A1994NB71800027
A1994NB80900015
A1994NB82100021
A1994NB82100022
A1994NB98500047
A1994NC03000004
A1994NC04300011
A1994NC04300023
A1994NC16500006
A1994NC18000009
A1994NC19300005
A1994NC39000002
A1994NC51900004
A1994NC52400013
A1994NC76900006
A1994NC77700004
A1994NC83700001
A1994ND12000063
A1994ND12100007
A1994ND15700006
A1994ND19400014
A1994ND22100007
A1994ND24600009
A1994ND28600001
A1994ND53600038
A1994ND60200005
A1994NE28600007
A1994NE75800004
A1994NF01700041
A1994NF02200001
A1994NF02200003
A1994NF67300014
A1994NF96800063
A1994NF96800119
A1994NG1

A1996UM67800006
A1996UM86400004
A1996UM88900049
A1996UM88900054
A1996UN12600008
A1996UN18900026
A1996UN25300082
A1996UN42300021
A1996UN47200047
A1996UN47400001
A1996UN47900048
A1996UN47900049
A1996UN49100005
A1996UN72500006
A1996UN96500005
A1996UP16800001
A1996UP25300007
A1996UP43200013
A1996UP43500059
A1996UP56800029
A1996UP62500015
A1996UP89900054
A1996UQ05500007
A1996UQ36600005
A1996UQ36600011
A1996UQ45500035
A1996UQ45500065
A1996UQ45500075
A1996UQ62000016
A1996UQ64900011
A1996UQ65700051
A1996UQ66000015
A1996UQ67800001
A1996UQ88100005
A1996UR09300034
A1996UR09300045
A1996UR09300052
A1996UR11200007
A1996UR13000075
A1996UR30000006
A1996UR30100004
A1996UR35800028
A1996UR41600014
A1996UR60400008
A1996UR97900041
A1996UR97900042
A1996UT10600094
A1996UT11000044
A1996UT24100007
A1996UT32300016
A1996UT32300019
A1996UT40100021
A1996UT40700021
A1996UT64100014
A1996UU11700002
A1996UU11800017
A1996UU28300022
A1996UU35600064
A1996UU36700005
A1996UU36700095
A1996UU41600004
A1996UU44300006
A1996UU6

,UT,citation_count,date
0,000070935900017,1,1998-11-15
1,000070935900017,2,1998-12-01
2,000070935900017,4,1998-12-15
3,000070935900017,5,1999-02-15
4,000070935900017,6,1999-03-15
5,000070935900017,9,1999-04-15
6,000070935900017,11,1999-05-15
7,000070935900017,13,1999-06-15
8,000070935900017,15,1999-08-15
9,000070935900017,17,1999-09-15


In [186]:
cont

9933

In [152]:
###or:
new_df=pd.read_pickle('/home/staff/julia/at_Northwestern/In_Text_Citations/In-Text-Citations-New/data/df_UT_dates_citation_counts.pickle')


In [9]:
data=[]

for UT in df_trajectories.UT.unique()[-25:]:
    
    df_plot=df_trajectories[df_trajectories['UT']==UT]

    trace = go.Scatter( x=df_plot['date'], 
                       y=df_plot['citation_count'], 
                       name=UT )
    
    data.append(trace)

    layout = go.Layout(
       # title='Plot Title',
        xaxis=dict(
            title='Date',
#             titlefont=dict(
#                 family='Courier New, monospace',
#                 size=18,
#                 color='#7f7f7f'
#             )
        ),
        yaxis=dict(
            title='# citations received',
#             titlefont=dict(
#                 family='Courier New, monospace',
#                 size=18,
#                 color='#7f7f7f'
#             )
        )
    )

fig = Figure(data=data, layout=layout)

    
    

iplot(data, filename='testing_time_series_citations.html')
   
offline.plot(fig, auto_open=True, image = 'png', image_filename="testing_time_series_citations" ,
                 output_type='file',  filename='/home/staff/julia/at_Northwestern/In_Text_Citations/In-Text-Citations-New/plots/testing_time_series_citations.html', validate=False)


'file:///home/staff/julia/at_Northwestern/In_Text_Citations/In-Text-Citations-New/plots/testing_time_series_citations.html'

40.0